In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
%pip install -q llama-index

Note: you may need to restart the kernel to use updated packages.


In [3]:
import  nest_asyncio

nest_asyncio.apply()

In [5]:
from llama_index.core import SimpleDirectoryReader

#loading
documents = SimpleDirectoryReader(input_files=['../src/The_7th_Sense_For_Data-Driven_Decision_Mastery_Paper_V2.pdf']).load_data()

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)


In [6]:
#splitting
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [7]:
#model
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model='gpt-4o-mini')
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002") 

In [8]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_store_index = VectorStoreIndex(nodes)

In [18]:
summary_query_engine = summary_index.as_query_engine(response_mode="tree_summarize", 
                                                     use_async=True,)
vector_store_query_engine = vector_store_index.as_query_engine()

In [19]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=("Useful for summarization questions related to the Given Document"),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_store_query_engine, 
    description=("Useful for retriving specific context from the given Document Paper"),
)

In [20]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool
    ],
    verbose=True
)

In [16]:
response = query_engine.query("What is 7th sense")
print(str(response))

Selecting query engine 1: The question 'What is 7th sense' requires retrieving specific context or information about the term from the given document..
The term "7th sense" in this context refers to an advanced capability for data-driven decision-making that leverages automated data analytics processes. It signifies a transformative approach that integrates artificial intelligence and machine learning techniques to enhance the analysis of large datasets, enabling decision-makers to derive insights quickly and accurately. This concept emphasizes the importance of automation in overcoming traditional data analysis challenges, ultimately driving strategic innovation and business success.


In [21]:
response = query_engine.query("What is summary")
print(str(response))

Selecting query engine 0: The question 'What is summary' relates to summarization, making choice 1 the most relevant..
The project focuses on automating the data analytics pipeline to enhance decision-making and drive strategic innovation in businesses. It connects to client databases to automate data preprocessing, cleaning, and imputation, ensuring accurate data types through advanced calculations. The system generates detailed abstracts that include metadata, feature descriptions, and statistical summaries, along with analyses like RFM, customer behavior clustering, and churn prediction.

Utilizing generative AI techniques, particularly retrieval-augmented generation (RAG) and few-shot prompting, the project converts text files into embeddings for efficient content retrieval. An interactive chatbox allows decision-makers to engage with the system, refining analysis outputs to align with business needs. This comprehensive approach empowers decision-makers with faster, more accurate i

In [22]:
print(len(response.source_nodes))

9
